
# Evaluation metrics for the Cityscapes dataset
In this Jupyter Notebook we show how to compute and visualize different evaluation metrics for the Cityscapes Pixel-Segmentation task

In [1]:
!pip install cityscapesscripts;

     |████████████████████████████████| 472 kB 11.4 MB/s eta 0:00:01
     |████████████████████████████████| 45 kB 11.5 MB/s eta 0:00:01
     |████████████████████████████████| 2.2 MB 15.9 MB/s eta 0:00:01
     |████████████████████████████████| 86 kB 16.7 MB/s eta 0:00:01


## Imports

In [1]:
import os
import torch
import zipfile
import numpy as np

import sklearn.metrics
from importlib import reload
from os.path import join as pjoin
from datasets.cityscapes import cityscapesDataset
from torchvision import transforms

In [2]:
local_path = "./Cityscapes"
pckgs_names = ["gtFine_trainvaltest.zip","leftImg8bit_trainvaltest.zip"]
dir_names = ["gtFine", "leftImg8bit"]

### Load the dataset
We assume that the datset *Cityscapes* has already been downloaded in the folder *local_path*. If this is not the case, check first the notebook [Exploring the Cityscapes dataset](./Cytyscapes_dataset.ipynb)

In [3]:
training_data = cityscapesDataset(local_path, split="train")

Annotations files processed


In [4]:
labels = training_data.label_ids()
labels

[-1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 255]

In [5]:
any(np.isin(np.array(labels), 45))

False

In [6]:
img, lbl = training_data[7]

In [7]:
lbl.size()

torch.Size([1024, 2048])

In [8]:
y_true = [training_data[i][1] for i in range(0,10)]

In [9]:
y_pred = [training_data[i][1] for i in range(15,25)]

## Evaluation report

In [27]:
import importlib
import evaluation
importlib.reload(evaluation)

<module 'evaluation' from '/project/simple-imageseg/evaluation.py'>

Create an evaluation report providing a dataloader and a model to make the predictions

Create an evaluation report providing the ground-truth values and the predictions.

In [28]:
evaluation = evaluation.EvaluationReport.from_predictions(y_true, y_pred, labels)

In [30]:
target_class = 2
metrics2 = evaluation.all_metrics(target_class)
metrics2

{'accuracy': 0.7431,
 'sensitivity': 0.6956,
 'specificity': 0.7579,
 'dice_coeff': 0.5626,
 'jaccard_sim': 0.3914,
 'f1_score': 0.5626}

The number of decimal places can be changed

In [43]:
evaluation.decimal_places = 2
evaluation.all_metrics(target_class)

{'accuracy': 0.74,
 'sensitivity': 0.7,
 'specificity': 0.76,
 'dice_coeff': 0.56,
 'jaccard_sim': 0.39,
 'f1_score': 0.56}

#### Sklearn check values

In [31]:
y_pred_b = []
y_true_b = [] 
for i in range(len(y_true)):
    y_true_b.append((y_true[i] == target_class).long().flatten().numpy())
    y_pred_b.append((y_pred[i] == target_class).long().flatten().numpy())

In [32]:
y_true_b  = np.concatenate(y_true_b, axis=None)
y_pred_b  = np.concatenate(y_pred_b, axis=None)

In [36]:
sk_accuracy = round(sklearn.metrics.accuracy_score(y_true_b, y_pred_b), 4)
sk_accuracy

0.7431

In [37]:
metrics2['accuracy'] == sk_accuracy

True

In [38]:
sk_js = round(sklearn.metrics.jaccard_score(y_true_b, y_pred_b), 4)
sk_js

0.3914

In [39]:
metrics2['jaccard_sim'] == sk_js

True

In [40]:
sk_f1 = round(sklearn.metrics.f1_score(y_true_b, y_pred_b), 4)
sk_f1

0.5626

In [41]:
metrics2['f1_score'] == sk_f1

True

In [42]:
metrics2['dice_coeff'] == sk_f1

True